## Задание 1

**Написать теггер на данных с руским языком**
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбмнации
2. написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
3. сравнить все реализованные методы сделать выводы


**Импорт библиотек**

In [1]:
import pyconll

import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.corpus import brown
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger

from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer, TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder

**Загрузка данных**

In [2]:
full_train = pyconll.load_from_file('./ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('./ru_syntagrus-ud-dev.conllu')

In [3]:
for sent in full_train[:2]:
    for token in sent:
        print(token.form, token.upos)
    print()

Анкета NOUN
. PUNCT

Начальник NOUN
областного ADJ
управления NOUN
связи NOUN
Семен PROPN
Еремеевич PROPN
был AUX
человек NOUN
простой ADJ
, PUNCT
приходил VERB
на ADP
работу NOUN
всегда ADV
вовремя ADV
, PUNCT
здоровался VERB
с ADP
секретаршей NOUN
за ADP
руку NOUN
и CCONJ
иногда ADV
даже PART
писал VERB
в ADP
стенгазету NOUN
заметки NOUN
под ADP
псевдонимом NOUN
" PUNCT
Муха NOUN
" PUNCT
. PUNCT



In [4]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    fdata_sent_test.append([token.form for token in sent])
    
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

Наибольшая длина предложения 205
Наибольшая длина токена 47


In [5]:
all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
all_test_texts = [' '.join(token.form for token in sent) for sent in full_test]

all_train_labels = [' '.join(token.form for token in sent) for sent in full_train]
all_test_labels = [' '.join(token.form for token in sent) for sent in full_test]
print('\n'.join(all_train_texts[:10]))

Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глубине стоял широкий письменный стол с бронзовыми чернильницами

**DefaultTagger**

In [6]:
default_tagger = nltk.DefaultTagger('NOUN')

default_tagger.evaluate(fdata_test)

0.23568564014423887

**UnigramTagger**

In [7]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

0.8772537323492737

**BigramTagger**

In [8]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

0.8829828463586425

**TrigramTagger**

In [9]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

0.882081353418933

In [10]:
trigram_tagger.tag(fdata_sent_test[2])

[('Это', 'PRON'),
 ('связано', 'VERB'),
 ('с', 'ADP'),
 ('тем', 'PRON'),
 (',', 'PUNCT'),
 ('что', 'SCONJ'),
 ('работа', 'NOUN'),
 ('каких-то', 'DET'),
 ('инструкций', 'NOUN'),
 ('алгоритма', None),
 ('может', 'VERB'),
 ('быть', 'AUX'),
 ('зависима', None),
 ('от', 'ADP'),
 ('других', 'ADJ'),
 ('инструкций', 'NOUN'),
 ('или', 'CCONJ'),
 ('результатов', 'NOUN'),
 ('их', 'DET'),
 ('работы', 'NOUN'),
 ('.', 'PUNCT')]

**Комбинация тэггеров**

In [11]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NOUN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test) 

0.9119991237825633

**Векторайзеры**

In [12]:
train_tok = []
train_label = []
for sent in fdata_train[:]:
    for tok in sent:
        train_tok.append(tok[0])
        train_label.append('NO_TAG' if tok[1] is None else tok[1])
        
test_tok = []
test_label = []
for sent in fdata_test[:]:
    for tok in sent:
        test_tok.append(tok[0])
        test_label.append('NO_TAG' if tok[1] is None else tok[1])

In [13]:
le = LabelEncoder()

train_enc_labels = le.fit_transform(train_label)
test_enc_labels = le.transform(test_label)

print(f'Classes: {le.classes_} \nNumber of classes: {len(le.classes_)}')

Classes: ['ADJ' 'ADP' 'ADV' 'AUX' 'CCONJ' 'DET' 'INTJ' 'NOUN' 'NO_TAG' 'NUM' 'PART'
 'PRON' 'PROPN' 'PUNCT' 'SCONJ' 'SYM' 'VERB' 'X'] 
Number of classes: 18


In [14]:
for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    scaler = StandardScaler(with_mean=False)
    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    

    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.fit_transform(X_test)    
    
    lr = LogisticRegression(random_state=0, max_iter = 100, n_jobs=7)
    lr.fit(X_train, train_enc_labels)

    pred = lr.predict(X_test)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

[ 7 13  1 ... 10 16 13]
<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.9436019276783608
[ 7 13  1 ... 10 16 13]
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9468961682337479
[ 7 13  1 ... 10 16 13]
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9489097833046878


In [24]:
params = {
    "booster": "gbtree",
    "objective": "multi:softmax",
    "num_class": 18,
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "max_depth": 25,
    "seed": 27
}

for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    coder = vectorizer(ngram_range=(1, 5), analyzer='char')
    
    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    dtrain = xgb.DMatrix(data=X_train, label=train_enc_labels)
    dtest = xgb.DMatrix(data=X_test, label=test_enc_labels)

    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=10,
        maximize=True)
    
    pred = model.predict(dtest)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.9303828396185084
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.9336855053415563
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.9332810972938361


In [16]:
from nltk.corpus import stopwords
stop_words = stopwords.words('russian')

In [17]:
params = {
    "booster": "gbtree",
    "objective": "multi:softmax",
    "num_class": 18,
    "eval_metric": "auc",
    "learning_rate": 0.1,
    "reg_lambda": 100,
    "max_depth": 4,
    "gamma": 10,
    "nthread": 6,
    "seed": 27
}

for vectorizer in [CountVectorizer, HashingVectorizer, TfidfVectorizer]:

    coder = vectorizer(ngram_range=(1, 5), analyzer='word', stop_words=stop_words)
    
    X_train = coder.fit_transform(train_tok)
    X_test = coder.transform(test_tok)
    
    dtrain = xgb.DMatrix(data=X_train, label=train_enc_labels)
    dtest = xgb.DMatrix(data=X_test, label=test_enc_labels)

    model = xgb.train(
        params=params,
        dtrain=dtrain,
        num_boost_round=10,
        maximize=True)
    
    pred = model.predict(dtest)

    print(vectorizer, accuracy_score(test_enc_labels, pred))

<class 'sklearn.feature_extraction.text.CountVectorizer'> 0.2569339129848684
<class 'sklearn.feature_extraction.text.HashingVectorizer'> 0.2569339129848684
<class 'sklearn.feature_extraction.text.TfidfVectorizer'> 0.2569339129848684


Из одиночных теггеров лучший результат показали биграмный (0.88298) и триграммный (0.88208) теггеры.

Комбинация теггеров  [UnigramTagger, BigramTagger, TrigramTagger] дает прибавку лучшему скору = 0.9119991237825633

Векторайзеры работают лучше теггеров. При этом хороший результат получается при анализе символьных н-грамм в сочетании с логистической регрессией. Лучший скор показывает TfidfVectorizer (0.9489). 

Бустинговые модели работают хуже. Но при увеличении глубины они показывают все лучший результат (хотя глубина 25 это совсем неправильно. Кажется, что для хороших результатов нужна еще предобработка и новые признаки). Вероятно, что подбор других параметров, регуляризации позволит добиться лучшего результата, чем логистическая регрессия.